2023/2/17<br>

This ocde is for getting retardation, azimuthal angle and MFA images based on PolSope algorithm.<br>
After conversion, all images are concatenated by Phase-only-correlated (POC) method.<br> 

In [9]:
import cv2
import itertools
import os, sys
import gc
import psutil
import math
import copy
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import skimage
from skimage.filters import threshold_otsu, threshold_triangle, rank
from skimage.util import img_as_ubyte
import module.Ret_MFA_func as Ret_MFA_func
from module.im_stitching import *
import multiprocessing as mp
from multiprocessing import Pool

In [10]:
#MFA coversion
#parameters of optical anistropy of cellulose
no=1.529 #refractive index of ordinary light
ne=1.599 #refractive index of extraordinary light
cellulose_ratio=0.45 #excerpt from the chemistry of solid wood

#
core_num=8
bg_file_num=4

In [11]:
current_path=os.getcwd()
image_path=os.path.join(current_path, "POM")
npz_path=os.path.join(current_path, "npz_file")
save_path=os.path.join(current_path, "fig_save")

if os.path.exists(save_path)==False:
    os.makedirs(save_path)

In [12]:
#list up all samples
samples_all=os.listdir(image_path)
samples_all=[i for i in samples_all if ".DS_Store" not in i]

#list up all samples already calculated
samples_calculated=os.listdir(os.path.join(npz_path, "conversion_results"))
samples_stitched=os.listdir(os.path.join(npz_path, "stitching_results"))

#get uncalculated samples
samples = list(set(samples_all)-set(samples_calculated))
samples_stitched = list(set(samples_all)-set(samples_stitched))

#find samlpe name from dierctory
thickness_list=[i.split("_")[-1] for i in samples_all]

Extract target image files

In [13]:
file_path_list=[]
sample_name_list=[]
sample_position_list=[]

for sample in tqdm(samples):
    sample_path=os.path.join(image_path, str(sample))
    
    file_path_list_temp=[]    
    for root, dirs, files in os.walk(os.path.join(image_path)):
        
        if (len(files)>0):
            file_path_extract=[os.path.join(root, files[i]) for i in range(len(files)) if sample in root]
            file_path_extract=[file_path for file_path in file_path_extract if not ".DS_Store" in file_path]
            
            file_path_list_temp.append(file_path_extract)
        else:
            pass
    
    file_path_list_temp=list(itertools.chain.from_iterable(file_path_list_temp))
    file_path_list.append(file_path_list_temp)
    
    sample_position=[file_path_list_temp[i].split("/")[-2].split("_")[-2] for i in range(len(file_path_list_temp)-bg_file_num)]
    sample_position_list.append(sample_position)

100%|██████████| 1/1 [00:00<00:00, 48.67it/s]


**Convert a pair of three images to a retardation, azimuthal angle and MFA image.**

In [14]:
#call class
polscope_func=Ret_MFA_func.Polscope_func
core_num=8


#ret, azimuthal angle & MFA conversion
#all conversion processes are done folder by folder.

for i in range(len(file_path_list)):
    #set target list
    target_list=file_path_list[i]
    im_list=[]
    
    for target in target_list:

        #load image files
        im=cv2.imread(target, -1)
        im=im/65535
        im_height, im_width=im.shape
        im=im.flatten()
        im_list.append(im)


    #set main and bg_im
    #bg_im and bg_setting_im
    im_bg_list=im_list[-1*(bg_file_num):]
    im_bg=im_bg_list[0]
    im_bg_setting=im_bg_list[1:]

    #main image
    im_main_list=im_list[:-1*(bg_file_num)]

    #set list
    ret_im_list_temp=[]
    ret_binary_im_list_temp=[]
    angle_im_list_temp=[]
    MFA_im_list_temp=[]


    #polscope conversion & MFA conversion
    for j in range(len(im_main_list)//3):
        im_setting=im_main_list[j*3:(j+1)*3]

        #define term A, B
        A=polscope_func.term_A_three(im_setting[0], im_setting[1], im_setting[2], im_bg)
        B=polscope_func.term_B_three(im_setting[0], im_setting[1], im_setting[2], im_bg)

        A_bg=polscope_func.term_A_three(im_bg_setting[0], im_bg_setting[1], im_bg_setting[2], im_bg)
        B_bg=polscope_func.term_B_three(im_bg_setting[0], im_bg_setting[1], im_bg_setting[2], im_bg)

        A_true=A-A_bg
        B_true=B-B_bg

        delta_array=[]
        phi_array=[]

        #multiprocessing
        #call class
        polscope_mp=Ret_MFA_func.Polscope_mp(core_num=core_num, term_A=A_true, term_B=B_true)


        ######################
        # Polscope converion #
        ######################

        #set core numbers
        with Pool(core_num) as p:
            #calculation MFA using cores 
            callback_ret = p.map(polscope_mp.calc_delta_three_mp, range(core_num))
            callback_phi = p.map(polscope_mp.calc_phi_three_mp, range(core_num))

        #combine results
        delta_array=[]
        phi_array=[]
        for k in range(len(callback_ret)):
            delta_array.extend(callback_ret[k])
            phi_array.extend(callback_phi[k])

        delta_array=np.asarray(delta_array)
        phi_array=np.asarray(phi_array)

        #replace nan to mean value
        delta_array=np.nan_to_num(delta_array, nan=np.mean(delta_array), copy=False)
        phi_array=np.nan_to_num(phi_array, nan=np.mean(phi_array), copy=False)
        ret_array=np.asarray(delta_array)*180/np.pi*546/360
        angle_array=np.asarray(phi_array)*180/np.pi

        
        #MFA convnversion
        d=float(thickness_list[i])*cellulose_ratio*1000

        #call class
        MFA_func=Ret_MFA_func.MFA_func_mp(core_num=core_num, target_array=ret_array, d=d)

        #################
        # MFA converion #
        #################

        #set core numbers
        with Pool(core_num) as p:
            #calculation MFA using cores 
            callback_MFA = p.map(MFA_func.MFA_loop_polscope, range(core_num))

        #combine results
        MFA_array=[]
        for k in range(len(callback_MFA)):
            MFA_array.extend(callback_MFA[k])

        MFA_array=np.asarray(MFA_array)

        #reshape all conversion results
        ret_mat=ret_array.reshape((im_height, im_width))
        angle_mat=angle_array.reshape((im_height, im_width))
        MFA_mat=MFA_array.reshape((im_height, im_width))

        #save result
        ret_im_list_temp.append(ret_mat)
        angle_im_list_temp.append(angle_mat)
        MFA_im_list_temp.append(MFA_mat)

        #binarization
        im=np.nan_to_num(ret_mat)
        threshold = threshold_triangle(im)
        img=np.where(im<threshold, 255, 0)
        ret_binary_im_list_temp.append(img)      
        
    #save them as npz
    #set path
    npz_temp_save_path=os.path.join(npz_path, "conversion_results", samples[i])

    if os.path.exists(npz_temp_save_path)==False:
        os.makedirs(npz_temp_save_path)

    np.savez_compressed(os.path.join(npz_temp_save_path, str(samples[i])+".npz"), 
                       ret_im_list=ret_im_list_temp, angle_im_list=angle_im_list_temp, MFA_im_list=MFA_im_list_temp, 
                       ret_binary_im_list=ret_binary_im_list_temp, sample_position_list=sample_position_list[i], 
                        sample_info=samples[i])

100%|██████████| 663552/663552 [00:04<00:00, 164650.93it/s]


100%|██████████| 663552/663552 [00:03<00:00, 171490.82it/s]


In [15]:
#delete list to release memory
try:
    del ret_im_list_temp, ret_binary_im_list_temp, angle_im_list_temp, MFA_im_list_temp
    del angle_array, delta_array, phi_array, ret_array, MFA_array
    del ret_mat, angle_mat, MFA_mat, img
    gc.collect()
except:
    pass

**Reload image files and stitch them**

In [16]:
#load npz file and all data sample by sample
for root, dirs, files in os.walk(os.path.join(npz_path, "conversion_results")):
    if len(files)>0:
        for file in files:
            if root.split("/")[-1] in samples_stitched:
                npz_file_path=os.path.join(root, file)
                npz_file=np.load(npz_file_path, allow_pickle=True)

                ret_im_list=npz_file["ret_im_list"]
                ret_binary_im_list=npz_file["ret_binary_im_list"]
                angle_im_list=npz_file["angle_im_list"]
                MFA_im_list=npz_file["MFA_im_list"]
                sample_position=npz_file["sample_position_list"]
                sample_info=npz_file["sample_info"]

                #image registration is done by two steps
                #1st step: images are stitched in the horizontal direction (result image has oblong shape)
                #2nd step: images generated in 1st step are sticthed in the longitudinal direction to make one large image.  


                #image registration
                height_div=int(sample_position[-1])+1 #number of rows

                #set list
                binary_im_stitch_list=[]
                ret_im_stitch_list=[]
                angle_im_stitch_list=[]
                MFA_im_stitch_list=[]

                ############
                # 1st step #
                ############

                for i in tqdm(range(height_div)):
                    #set target image. process is done row by row
                    target_im_binary_list=list(ret_binary_im_list[i*len(ret_binary_im_list)//height_div : 
                                                                  (i+1)*len(ret_binary_im_list)//height_div])
                    target_im_ret_list=list(ret_im_list[i*len(ret_im_list)//height_div : (i+1)*len(ret_im_list)//height_div])
                    target_im_angle_list=list(angle_im_list[i*len(angle_im_list)//height_div : (i+1)*len(angle_im_list)//height_div])
                    target_im_MFA_list=list(MFA_im_list[i*len(MFA_im_list)//height_div : (i+1)*len(MFA_im_list)//height_div])

                    #if only one image exists in each row, first step is skipped.
                    if len(target_im_binary_list)<2:
                        binary_im_stitch_list.append(target_im_binary_list[0])
                        ret_im_stitch_list.append(target_im_ret_list[0])
                        angle_im_stitch_list.append(target_im_angle_list[0])
                        MFA_im_stitch_list.append(target_im_MFA_list[0])
                        continue

                    binary_im_stitch, ret_im_stitch = stitch_im_POC(target_im_binary_list, target_im_ret_list)
                    _, angle_im_stitch = stitch_im_POC(target_im_binary_list, target_im_angle_list)
                    _, MFA_im_stitch = stitch_im_POC(target_im_binary_list, target_im_MFA_list)


                    binary_im_stitch_list.append(binary_im_stitch)
                    ret_im_stitch_list.append(ret_im_stitch)
                    angle_im_stitch_list.append(angle_im_stitch)
                    MFA_im_stitch_list.append(MFA_im_stitch)

                ############
                # 2nd step #
                ############

                binary_image_result, ret_image_result = stitch_im_POC(binary_im_stitch_list, ret_im_stitch_list)
                _, angle_image_result = stitch_im_POC(binary_im_stitch_list, angle_im_stitch_list)
                _, MFA_image_result = stitch_im_POC(binary_im_stitch_list, MFA_im_stitch_list)

                #set path
                stitch_im_save_path=os.path.join(save_path, "stitching_results")
                if os.path.exists(stitch_im_save_path)==False:
                    os.makedirs(stitch_im_save_path)

                #save result as image
                fig, ax=plt.subplots(1, 3, figsize=(17, 4))
                im1=ax[0].imshow(ret_image_result, cmap="gist_ncar")
                pp=fig.colorbar(im1, ax=ax[0])
                pp.mappable.set_clim(0, 137)

                im2=ax[1].imshow(angle_image_result, cmap="jet")
                pp=fig.colorbar(im2, ax=ax[1])
                pp.mappable.set_clim(0, 180)

                im3=ax[2].imshow(MFA_image_result, cmap="gist_ncar")
                pp=fig.colorbar(im3, ax=ax[2])
                pp.mappable.set_clim(0, 60)
                plt.subplots_adjust(wspace=0.1)
                plt.tight_layout()
                plt.savefig(os.path.join(stitch_im_save_path, str(os.path.splitext(os.path.basename(npz_file_path))[0])+".png"), dpi=450)
                plt.close()

                #save result as npz
                stitch_save_path=os.path.join(npz_path, "stitching_results", str(sample_info))

                if os.path.exists(stitch_save_path)==False:
                    os.makedirs(stitch_save_path)

                np.savez_compressed(os.path.join(stitch_save_path, str(sample_info)+".npz"),
                                   ret_binary_im=binary_image_result, ret_im=ret_image_result, 
                                   angle_im=angle_image_result, MFA_im=MFA_image_result, 
                                   sample_info=sample_info)
            

100%|██████████| 3/3 [00:38<00:00, 12.89s/it]
